This notebook contains the steps I followed to create a correct G drive

In [1]:
import geopandas as gpd
import pandas as pd

edges = gpd.read_file('../geojson/gipuzkoa_drive_edges.geojson')
nodes = gpd.read_file('../geojson/gipuzkoa_drive_nodes.geojson')

print('Edges:', edges.crs)
print('Nodes:', nodes.crs)

Edges: EPSG:4326
Nodes: EPSG:4326


In [2]:
print(nodes.head())
print(edges.head())

    osmid          y         x  street_count highway   ref  \
0  448001  43.179563 -2.489983             3    None  None   
1  448006  43.182655 -2.481422             4    None  None   
2  451289  43.192781 -2.438172             3    None  None   
3  469503  43.191805 -2.451204             3    None  None   
4  469506  43.191796 -2.452494             3    None  None   

                    geometry  
0  POINT (-2.48998 43.17956)  
1  POINT (-2.48142 43.18266)  
2  POINT (-2.43817 43.19278)  
3    POINT (-2.4512 43.1918)  
4   POINT (-2.45249 43.1918)  
        u           v  key                            osmid  oneway reversed  \
0  448001  3934729820    0                         44716332   False    False   
1  448001      916343    0                       1009087270   False     True   
2  448006    25433518    0                         21770157   False    False   
3  448006   299660335    0                         44580274    True    False   
4  451289  5674243805    0  [ 71321937, 4

In [3]:
from shapely.geometry import Point

# Create start and end points for each edge
edges['start_point'] = edges['geometry'].apply(lambda x: Point(x.coords[0]))
edges['end_point'] = edges['geometry'].apply(lambda x: Point(x.coords[-1]))

# Check the resulting columns
print(edges.head())

        u           v  key                            osmid  oneway reversed  \
0  448001  3934729820    0                         44716332   False    False   
1  448001      916343    0                       1009087270   False     True   
2  448006    25433518    0                         21770157   False    False   
3  448006   299660335    0                         44580274    True    False   
4  451289  5674243805    0  [ 71321937, 4358223, 10672063 ]   False     True   

    length junction  maxspeed     time_s  \
0  113.604     None        30  13.632480   
1   42.152     None        30   5.058240   
2   77.905     None        30   9.348600   
3  229.730     None        30  27.567600   
4  907.995     None        70  46.696886   

                                            geometry  \
0  LINESTRING (-2.48998 43.17956, -2.48987 43.179...   
1  LINESTRING (-2.48998 43.17956, -2.49014 43.179...   
2  LINESTRING (-2.48142 43.18266, -2.48134 43.182...   
3  LINESTRING (-2.48142 43.182

In [4]:
from scipy.spatial import cKDTree
import numpy as np

# Build the cKDTree for the nodes (for efficient nearest-neighbor search)
nodes['coords'] = list(zip(nodes.geometry.x, nodes.geometry.y))
node_tree = cKDTree(np.array(nodes['coords'].tolist()))

# Function to find nearest node
def get_nearest_node(point):
    dist, index = node_tree.query([point.x, point.y])
    return nodes.iloc[index]['osmid']

# Find the nearest node for each start and end point
edges['nearest_node_start'] = edges['start_point'].apply(get_nearest_node)
edges['nearest_node_end'] = edges['end_point'].apply(get_nearest_node)

In [5]:
print(edges.columns)

Index(['u', 'v', 'key', 'osmid', 'oneway', 'reversed', 'length', 'junction',
       'maxspeed', 'time_s', 'geometry', 'start_point', 'end_point',
       'nearest_node_start', 'nearest_node_end'],
      dtype='object')


In [46]:
# drop edges start and end points
edges = edges.drop(columns=['start_point', 'end_point'])
edges.to_file('../geojson/gipuzkoa_drive_edges_nearest_nodes.geojson', driver='GeoJSON')
nodes.to_file('../geojson/gipuzkoa_drive_nodes_nearest_nodes.geojson', driver='GeoJSON')

In [6]:
import networkx as nx

# Create a graph from your edges and nodes
G = nx.DiGraph()  # Directed graph if the roads are one-way, or nx.Graph() for undirected

# Add nodes to the graph
for _, row in nodes.iterrows():
    G.add_node(row['osmid'], x=row['x'], y=row['y'])

# Add edges with multiple attributes
for _, row in edges.iterrows():
    u = row['nearest_node_start']
    v = row['nearest_node_end']
    time = row['time_s']
    length = row['length']
    maxspeed = row['maxspeed']
    geometry = row['geometry']
    
    # Add edge with time, length, maxspeed, and geometry
    G.add_edge(u, v, weight=time, length=length, maxspeed=maxspeed, geometry=geometry)

In [16]:
o_lon, o_lat=-2.01578381790964,43.304574686941336
#o_lon, o_lat=-1.9820539544817326,43.3196830982974
d_lon, d_lat= -1.9687977669669718, 43.29232181880548

# get nearest nodes
orig_node = get_nearest_node(Point(o_lon, o_lat))
dest_node = get_nearest_node(Point(d_lon, d_lat))

In [17]:
# find the shortest path
route = nx.shortest_path(G, orig_node, dest_node, weight='time_s')

# Get the geometry of the shortest path
route_geom = []
for i in range(len(route)-1):
    u = route[i]
    v = route[i+1]
    route_geom.append(G[u][v]['geometry'])

# Create a GeoDataFrame from the route geometry
route_gdf = gpd.GeoDataFrame({'geometry': route_geom}, crs=edges.crs)

# save the route to a file
#route_gdf.to_file('../geojson/shortest_path.geojson', driver='GeoJSON')

# show total time
total_time = sum([G[u][v]['weight'] for u, v in zip(route[:-1], route[1:])])
print('Total time:', total_time, 'seconds')

# time to min (would add a 1.25 coef to account for the average speed, as it is not always the max speed)
total_time = total_time/60
print('Total time:', 1.25*total_time, 'minutes')

Total time: 516.494292 seconds
Total time: 10.76029775 minutes
